In [1]:
# imports
import sklearn 
import requests
import pandas
import sys
import base64
import re

Create REST API calls to server to fetch operational data and current system conditions, store in df.

In [2]:
# fetch past operational data
op_url = 'http://localhost:8888/api/v2/operations'
headers = {'Accept': 'application/json', 'KEY' :'ADMIN123'}

op_response = requests.get(op_url, headers=headers)
op_data = pandas.DataFrame(op_response.json())
op_data = op_data.reset_index()  # make sure indexes pair with number of rows

# fetch current system conditions of active agent
# NOTE: using first trusted agent by default, replace with valid agent(s) for operation + alive and trusted
agents_url = 'http://localhost:8888/api/v2/agents'

agents_response = requests.get(agents_url, headers=headers)
agents_list = agents_response.json()
# select trusted agent
agent_selected = None
for agent in agents_list:
    if agent["trusted"] == True:
        # TODO: insert check for whether agent is alive
        agent_selected = agent

if agent_selected == None:
    print("FAILURE TO FIND AGENT")

FAILURE TO FIND AGENT


Select and examine useful features of operations for Naive Bayes Model.

In [3]:
# print all traits of link facts
sources_url = 'http://localhost:8888/api/v2/sources'
headers = {'Accept': 'application/json', 'KEY' :'ADMIN123'}

source = requests.get(sources_url, headers=headers).json()
sources = str(requests.get(sources_url, headers=headers).json())

# print(type(sources))
# print(sources)

# fact traits
fact_traits = set()


x = re.findall("(?<=\'trait': )'.*?\'", sources)
for match in x:
    fact_traits.add(match.replace("'", ""))

fact_traits

{'',
 'domain.user.name',
 'file.last.accessed',
 'file.last.modified',
 'file.sensitive.content',
 'file.sensitive.extension',
 'host.archive.password',
 'host.dir.compress',
 'host.dir.staged',
 'host.file.path',
 'host.user.name',
 'linux.excluded.directories',
 'linux.included.directories',
 'linux.included.extensions',
 'linux.staging.location',
 'pseudo.data.identifier',
 'remote.host.ip',
 'remote.ssh.ip',
 'safe.mode.enabled',
 'server.malicious.url',
 'windows.excluded.directories',
 'windows.excluded.extensions',
 'windows.included.directories',
 'windows.included.extensions',
 'windows.staging.location'}

In [7]:
# operation object cotains variety of interesting features
# chain contains all operation data/success (majority interest)
# print(thief_win_op["chain"][0])
# print(thief_win_op["adversary"])
import base64
print(base64.b64decode(thief_win_op["chain"][0]["command"]))


b'New-Item -Path "." -Name "staged" -ItemType "directory" -Force | foreach {$_.FullName} | Select-Object'


In [8]:
# link success df

used_fact_traits = set()
# for each operation
for index, cur_op in op_data.iterrows():
    # save relevant op info
    
    # run through link/ability chain within operation
    for cur_link in cur_op["chain"]:
        # save relevant link
        print("LINK")
        print(base64.b64decode(cur_link["command"]))
        print(cur_link["ability"]["ability_id"])
        print(cur_link["executor"]["command"])
        if(len(cur_link["used"])> 0):
            for used_fact in cur_link["used"]:
                print("~~~~~~~~~~~  fact (val,trait):")
                print(used_fact["value"])
                print(used_fact["trait"])
                used_fact_traits.add(used_fact["trait"])
        print("************************")
print(used_fact_traits)

# iterate through abilities from api
# print used facts

LINK
b"find / -name '*.png' -type f -not -path '*/\\.*' -size -500k 2>/dev/null | head -5"
90c2efaa-8205-480d-8bb6-61d90dbaf81b
find / -name '*.#{file.sensitive.extension}' -type f -not -path '*/\.*' -size -500k 2>/dev/null | head -5
~~~~~~~~~~~  fact (val,trait):
png
file.sensitive.extension
************************
LINK
b"find / -name '*.yml' -type f -not -path '*/\\.*' -size -500k 2>/dev/null | head -5"
90c2efaa-8205-480d-8bb6-61d90dbaf81b
find / -name '*.#{file.sensitive.extension}' -type f -not -path '*/\.*' -size -500k 2>/dev/null | head -5
~~~~~~~~~~~  fact (val,trait):
yml
file.sensitive.extension
************************
LINK
b"find / -name '*.wav' -type f -not -path '*/\\.*' -size -500k 2>/dev/null | head -5"
90c2efaa-8205-480d-8bb6-61d90dbaf81b
find / -name '*.#{file.sensitive.extension}' -type f -not -path '*/\.*' -size -500k 2>/dev/null | head -5
~~~~~~~~~~~  fact (val,trait):
wav
file.sensitive.extension
************************
LINK
b'whoami'
c0da588f-79f0-4263-8998-7496

In [4]:
import sys
# link success df

# for each operation
for index, cur_op in op_data.iterrows():
    # save relevant op info
    
    # run through link/ability chain within operation
    for cur_link in cur_op["chain"]:
        # save relevant link
        print("LINK")
        print(base64.b64decode(bytes(cur_link["command"], 'utf-8')))
        print(cur_link["ability"]["ability_id"])
        print(cur_link["executor"]["command"])
        if(len(cur_link["used"])> 0):
            for used_fact in cur_link["used"]:
                print(used_fact["value"])
        print("************************")
#         for items in cur_link:
#             print(items)
#         print(cur_link["ability"])


LINK


NameError: name 'base64' is not defined

Build up df of past links from Operations

In [5]:
# store link info in lists
# later convert to df, for efficiency
statuses = []
ability_ids = []
usable_facts = [] # contains lists of fact dicts with 0 or more items
host_platforms = []
host_protocols = []

planners = []
host_trusted_statuses = []
host_contacts = []
host_architectures = []
host_privileges = []
obfuscators = []
adversary_ids = []



# for each operation
for index, cur_op in op_data.iterrows():
    
    # run through each link chain within operation
    for cur_link in cur_op["chain"]:
    
        # save relevant global op info
        # NOTE: Temporarily using first agent in host_group list
        host_platforms.append(cur_op["host_group"][0]["platform"])
        host_protocols.append(cur_op["host_group"][0]["contact"])
        
        # save relevant link info
        ability_ids.append(cur_link["ability"]["ability_id"])
        statuses.append(cur_link["status"])
        cur_used_global_facts = {} # key: trait, val: value    
        
        if(len(cur_link["used"]) > 0):
            # iterate through facts
            for used_fact in cur_link["used"]:
                # if fact is not unique to host
                if not used_fact["trait"].startswith("host."):
                    # save fact
                    cur_used_global_facts[used_fact["trait"]] = used_fact["value"]

        # save current usable facts
        usable_facts.append(cur_used_global_facts)
        
# TODO:
# flag implementation for querying by more features

# create link success df from lists of data
data_link_success = {
    "Status": statuses,
     "Ability_ID": ability_ids, 
     "Link_Facts": usable_facts, 
     "Platform": host_platforms, 
     "Protocol": host_protocols
    }

print(len(statuses))
print(len(ability_ids))
print(len(usable_facts))
print(len(host_platforms))
print(len(host_protocols))
link_success_df = pandas.DataFrame(data_link_success)
link_success_df

704
704
704
704
704


,Status,Ability_ID,Link_Facts,Platform,Protocol
0,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'png'},linux,HTTP
1,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'yml'},linux,HTTP
2,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'wav'},linux,HTTP
3,0,c0da588f-79f0-4263-8998-7496b1a40596,{},linux,HTTP
4,0,c1cd6388-3ced-48c7-a511-0434c6ba8f48,{},linux,HTTP
...,...,...,...,...,...
699,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'yml'},windows,tcp
700,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'png'},windows,tcp
701,1,300157e5-f4ad-4569-b533-9d1fa0e74d74,{},windows,tcp
702,1,300157e5-f4ad-4569-b533-9d1fa0e74d74,{},windows,tcp


Train model with Scikit (NB, RF, KNN)

In [12]:
from sklearn.naive_bayes import GaussianNB


gnb = sklearn.naive_bayes.GaussianNB()

Create link probability function (NB formula)

P(A|B) = $\frac{P(B|A)*P(A)}{P(B)}$

example call:
calculate P(status=0 | ability=x, fact=z, platform=linux, protocol=TCP)

=$\frac{P(ability=x, fact=z, platform=linux, protocol=TCP | status=0) * P(status=0)}{P(ability=x, fact=z, platform=linux, protocol=TCP)}$

Method call: 

NBLinkSuccessProb("90c2efaa-8205-480d-8bb6-61d90dbaf81b", {'file.sensitive.extension': 'wav'}, "windows", "tcp")

In [26]:
def NBLinkSuccessProb(ability_id, fact_dict, host, protocol):
#     print(ability_id)
#     print("Ability_ID == '" + str(ability_id) +"' and Platform == '" + host +"' and Protocol == '" + protocol +"'")
    # query by ability and host characteristics
    query_df = link_success_df.query("Ability_ID == '" + str(ability_id) +"' and Platform == '" + host + "' and Protocol == '" + protocol +"'")
    # query by facts
    for fact_key in fact_dict:
        pass
#     print (100 * query_df['Status'].value_counts(normalize=True))
    return query_df
    
    

NBLinkSuccessProb("90c2efaa-8205-480d-8bb6-61d90dbaf81b", {'file.sensitive.extension': 'wav'}, "windows", "tcp")

,Status,Ability_ID,Link Facts,Platform,Protocol
391,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'wav'},windows,tcp
392,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'yml'},windows,tcp
393,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'png'},windows,tcp
398,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'wav'},windows,tcp
399,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'yml'},windows,tcp
400,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'png'},windows,tcp
405,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'wav'},windows,tcp
406,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'yml'},windows,tcp
407,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'png'},windows,tcp
412,0,90c2efaa-8205-480d-8bb6-61d90dbaf81b,{'file.sensitive.extension': 'wav'},windows,tcp


Create link probability function (Scikit Model)